HistGradientBoosting is a conveneint scikit-learn version of the Gradient Boosting Algorithm. It is modeled after LightGBM. Even though it doesn't have nearly as maney features as otehr GBM libraries, it is simple to use and oftentimes on par with those more powerful libraries in terms of predictive performance 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
import glob
import os
import gc

In [ ]:
train = pd.read_csv('../input/ncaaw-2021-features-only/tourney.csv')
test = pd.read_csv('../input/ncaaw-2021-features-only/test.csv')

In [ ]:
y = train["result"]
s = train["Season"]
X = train.drop(['Season','TeamID1','TeamID2','result'], axis=1)


X_test = test.drop(['ID', 'Season','TeamID1','TeamID2'], axis=1)

In [ ]:
train_oof = np.zeros((X.shape[0],))
test_preds = 0
train_oof.shape

In [ ]:
NUM_FOLDS = 10
kf = GroupKFold(n_splits=NUM_FOLDS)
max_iter = 550

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(X, y, s))):
        #print(f'Fold {f}')
        train_df, val_df = X.iloc[train_ind], X.iloc[val_ind]
        train_target, val_target = y.iloc[train_ind], y.iloc[val_ind]
        
        
        model = HistGradientBoostingClassifier(max_iter=max_iter, validation_fraction=None, learning_rate=0.01, max_depth=2, min_samples_leaf=32)
        

        model =  model.fit(train_df, train_target)
        temp_oof = model.predict_proba(val_df)[:,1]
        temp_test = model.predict_proba(X_test)[:,1]

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(log_loss(val_target, temp_oof))

In [ ]:
log_loss(y, train_oof)


In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
test = pd.read_csv('../input/ncaaw-2021-features-only/test.csv')
WSampleSubmission = pd.read_csv('../input/ncaaw-march-mania-2021/WSampleSubmissionStage1.csv')

In [ ]:
idx = test_preds.shape[0] //2
test_preds[idx:] = 1 - test_preds[idx:]

pred = pd.concat([test.ID, pd.Series(test_preds)], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub = WSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
sub.to_csv('submission.csv', index=False)
sub.head()